In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import re
import random
from scipy.stats import pearsonr

In [2]:
FACE_MESH = pd.read_csv('mediaPipe/faceMesh/face_mesh.csv')

In [7]:
def landmarks_mapper(video: str):
    
    OPENFACE_DIR = 'openFace/processed/'
    MEDIAPIPE_DIR = 'mediaPipe/processed/'
    OPENFACE_LANDMARKS = [i for i in range(0, 68)]
    
    of_df = pd.read_csv(OPENFACE_DIR + video + '_openface.csv')
    mp_df = pd.read_csv(MEDIAPIPE_DIR + video + '_mediapipe.csv')
    
    final_df = of_df['frame']
    for i, landmark in enumerate(OPENFACE_LANDMARKS):

        mask = (FACE_MESH['openFace_landmark'] == landmark) # group the mediapipe landmarks by the openface landmark

        tmp1_df = of_df[[f'x_{landmark}', f'y_{landmark}']]
        tmp1_df.columns = [f'openface_x_{landmark}', f'openface_y_{landmark}']
        
        mediaPipe_landmarks_X = ['x_'+str(ID) for ID in FACE_MESH[mask].id] # collect the X coordinate
        mediaPipe_landmarks_Y = ['y_'+str(ID) for ID in FACE_MESH[mask].id] # collect the Y coordinate
        
        tmp2_df = pd.DataFrame({f'mediapipe_x_{landmark}':mp_df[mediaPipe_landmarks_X].mean(axis=1),
                                f'mediapipe_y_{landmark}':mp_df[mediaPipe_landmarks_Y].mean(axis=1)})
        
        final_df = pd.concat([final_df, tmp1_df, tmp2_df], axis=1)
        
    return final_df

In [4]:
def pearson(df: pd.DataFrame):
    
    pearson_lndmk = []
    for i in range(68):
        
        pearson_X = pearsonr(df[f'openface_x_{i}'].tolist(), df[f'mediapipe_x_{i}'].tolist())[0]
        pearson_Y = pearsonr(df[f'openface_y_{i}'].tolist(), df[f'mediapipe_y_{i}'].tolist())[0]
        pearson_lndmk.append((pearson_X+pearson_Y)/2)
    
    pearson_video = np.mean(pearson_lndmk)
    
    return pearson_video

In [16]:
videos = []
pearson_videos = []
for video in os.listdir('dataset/ElderReact_Data/ElderReact_train/'):
    
    video = video[:-4]
    
    videos.append(video)
    
    df = landmarks_mapper(video)
    pearson_videos.append(pearson(landmarks_mapper(video)))

pearson_dataset = pd.DataFrame({'video': videos, 'pearson': pearson_videos})
%store pearson_dataset
pearson_dataset

,video,pearson
0,whack_your_boss_12,0.974169
1,happy_wheels_54,0.968697
2,would_you_rather_29,0.983525
3,play_red_dead_138,0.763068
4,Branda_schmitz_116,0.987913
...,...,...
610,bully_burger_54,0.980151
611,game_of_thrones_16,0.991651
612,qwop_38,0.978327
613,cuphead_58,0.985006


[-0.42217233828076656,
 0.04053348632970934,
 0.05174933146657267,
 0.2451964856589686,
 0.27406683737105064,
 0.4378453488012905,
 0.5084017250700633,
 0.5278153741809202,
 0.5627180407367525,
 0.5873287050487643,
 0.6271127019066324,
 0.6665276412576308,
 0.6881151970082522,
 0.6961409136426976,
 0.7015614353919573,
 0.707041836189073,
 0.715192142556615,
 0.7225765015876895,
 0.749711553370716,
 0.7497276327737339,
 0.7630681244769191,
 0.7663274163755179,
 0.7770408227942562,
 0.7877198430420111,
 0.789928452400751,
 0.7928078477680573,
 0.7929266592259021,
 0.7981352069627499,
 0.8051033648408383,
 0.8132798520472195,
 0.8140135065046901,
 0.8145835706340144,
 0.8171482254595256,
 0.8174795769122032,
 0.8200875281014568,
 0.8201070205359924,
 0.8204966847651325,
 0.8269637203676482,
 0.8308190425234221,
 0.8317516797039041,
 0.8327813268611743,
 0.8380021456719926,
 0.8381790771700569,
 0.8442440230798574,
 0.8452603128851699,
 0.8455445647834943,
 0.8494070444621921,
 0.850337516